# SpaceX Mission Data Extraction and Cleaning

In [ ]:
import requests
from bs4 import BeautifulSoup

#Step 1: Web Scraping SpaceX Mission Launch Data
url = 'https://www.spacex.com/launches/'
response = requests.get(url)
soup1 = BeautifulSoup(response.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
missions = soup2.find_all('div', class_="item")
mission_data = []
for mission in missions:
    launch_date = mission.find('div',class_='date').text.strip()
    mission_name = mission.find('div',class_='label').text.strip()
    article = mission.find('a', class_='article-header', attrs={'aria-hidden': 'true'})
    if article:
        link = article.get('href')
    mission_data.append({
        'Mission': mission_name,
        'Launch Date': launch_date,
        'launch_link': link
    })
print(mission_data)

In [ ]:
# Step 2: Extracting Mission Headlines from each mission launch link
for mission in mission_data:
    link = mission['launch_link']
    if link and link != 'N/A':
        mission_url = f"https://www.spacex.com{link}"
        mission_response = requests.get(mission_url)#get each link page contents
        mission_soup = BeautifulSoup(mission_response.content, 'html.parser')#parse the page content with BeautifulSoup
    #List to hold the extracted headlines
        mission_headline = []
        
    #Find the container with the paragraph tags(p)
        mission_headlines = mission_soup.find('div', id='scroller')
        
    #Initialize a counter
        counter = 0
        
    #Iterate through the paragraph tags(p) in the container
        for news in mission_headlines.find_all('p'):
            
        #Append the paragraph text to the list
            mission_headline.append(news.get_text(strip=True))
            
        #Increase the counter
            counter += 2
            
        #Break loop after first 2 paragraphs(p)
            if counter >= 3:
                break
                
        #Add list to mission_data
        mission['mission_headline'] = mission_headline
for data in mission_data:
    print(data)

In [ ]:
#Converting to DataFrame
import pandas as pd
df = pd.DataFrame(mission_data, columns=['Mission','Launch Date', 'launch_link', 'mission_headline'])
df

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df

In [ ]:
#Step 3: Cleaning the extracted mission headlines 
df['mission_headline'] = df['mission_headline'].str.replace("[",'')
df['mission_headline'] = df['mission_headline'].str.replace("]",'')
df["mission_headline"] = df['mission_headline'].str.replace("{",'')
df['mission_headline'] = df['mission_headline'].str.replace("}",'')
df["mission_headline"] = df['mission_headline'].str.replace("'",'')
df["mission_headline"] = df["mission_headline"].str.replace("'new':",'')
df["mission_headline"] = df['mission_headline'].str.replace('"','')
df

In [ ]:
#Dropping Columns
df = df.drop(columns = "Unnamed: 0")
df

In [ ]:
#Save to csv
df.to_csv('Spacex_Mission_Launch_data.csv')